In [1]:
#!/usr/bin/env python3
# -*-coding:utf-8 -*-
# =============================================================================
"""
@Author        :   Yujie He
@File          :   gen_tracking_res.py
@Date created  :   2021/10/19
@Maintainer    :   Yujie He
@Email         :   yujie.he@epfl.ch
"""
# =============================================================================
"""
The module provides pedestrian tracking results from detection results based on
AB3DMOT framework
"""
# =============================================================================


import os
import sys

sys.path.append("/home/jupyter-dominik/AB3DMOT")
sys.path.append("/home/jupyter-dominik/AB3DMOT/Xinshuo_PyToolbox")
import argparse
import numpy as np

from qolo.core.crowdbot_data import CrowdBotDatabase
from AB3DMOT_libs.model import AB3DMOT

/home/jupyter-dominik/AB3DMOT/AB3DMOT_libs/kitti_oxts.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def rotx(t):
/home/jupyter-dominik/AB3DMOT/AB3DMOT_libs/kitti_oxts.py:15: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def roty(t):
/home/jupyter-dominik/AB3DMOT/AB3DMOT_libs/kitti_oxts.py:24: NumbaDeprecationWarning: The 'nopython' keyword argument was not

In [2]:
def reorder(boxes):
    # from x, y, z,  l,  w, h, theta (lidar frame: x-forward, y-left, z-up)
    # to   h, w, l, -y, -z, x, theta (cam frame: z-forward, x-right, y-down)
    inds = [5, 4, 3, 1, 2, 0, 6]
    boxes = boxes[:, inds]
    boxes[:, 3] *= -1
    boxes[:, 4] *= -1
    return boxes


def reorder_back(boxes):
    # from h, w, l, -y, -z, x, theta, ID
    # to   x, y, z,  l,  w, h, theta, ID
    inds = [5, 3, 4, 2, 1, 0, 6, 7]
    boxes = boxes[:, inds]
    boxes[:, 1] *= -1
    boxes[:, 2] *= -1
    return boxes


if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description="convert data from rosbag")

    # parser.add_argument(
    #     "-f",
    #     "--folder",
    #     default="0424_mds_test",
    #     type=str,
    #     help="different subfolder in rosbag/ dir",
    # )
    # parser.add_argument(
    #     "--overwrite",
    #     dest="overwrite",
    #     action="store_true",
    #     help="Whether to overwrite existing output (default: false)",
    # )
    # parser.set_defaults(overwrite=False)
    # parser.add_argument(
    #     "--save_raw",
    #     dest="save_raw",
    #     action="store_true",
    #     help="Whether to save raw data of detection results (default: false)",
    # )
    # parser.set_defaults(save_raw=False)
    # args = parser.parse_args()

    class cfg:
        def __init__(self, folder, overwrite, save_raw):
            self.folder = folder
            self.overwrite = overwrite
            self.save_raw = save_raw

    args = cfg(folder="0424_mds_test", overwrite=True, save_raw=False)
        
    
    min_conf = 0.7

    #SERVER
    config_path = '/scai_data/data01/daav/CrowdBot/config/data_path.yaml'

    cb_data = CrowdBotDatabase(args.folder, config=config_path)

    counter = 0
    seq_num = cb_data.nr_seqs()

    for seq_idx in range(seq_num):
        seq = cb_data.seqs[seq_idx]

        counter += 1
        print("({}/{}): {} frames".format(counter, seq_num, cb_data.nr_frames(seq_idx)))

        # seq dest: data/xxxx_processed/alg_res/tracks/seq
        if args.save_raw:
            trk_seq_dir = os.path.join(cb_data.trks_dir, seq)
        tnpy_all_path = os.path.join(cb_data.trks_dir, seq + '.npy')

        if not os.path.exists(tnpy_all_path) or args.overwrite:
            out_trk_all = dict()

            # tracker = AB3DMOT(max_age=2, min_hits=3)
            log = os.path.join(cb_data.alg_res_dir, "logs", seq + '.txt')
            tracker = AB3DMOT(max_age=20, min_hits=5, thres=0.2, log=log)  # lidar is about 10 FPS

            for fr_idx in range(cb_data.nr_frames(seq_idx)):
                _, dets, dets_conf, _ = cb_data[seq_idx, fr_idx]

                dets = dets[dets_conf > min_conf]
                dets = reorder(dets)
                
                trk_input = {"dets": dets, "info": np.zeros_like(dets)}
                trks = tracker.track(trk_input, fr_idx, cb_data.seqs[seq_idx])[0][0]
                trks = reorder_back(trks)
                
                if fr_idx%50 == 0:
                    print("DETS SHAPE = " + str(dets.shape))
                    print("TRKS SHAPE = " + str(trks.shape))

                if fr_idx == 0:
                    # find the smallest id and normalize to 1
                    min_id = min(trks[:, -1])
                    offset = min_id - 1
                trks[:, -1] = trks[:, -1] - offset

                if args.save_raw:
                    f_path = os.path.join(
                        trk_seq_dir,
                        cb_data.frames[seq_idx][fr_idx].replace("nby", "txt"),
                    )
                    os.makedirs(trk_seq_dir, exist_ok=True)
                    np.savetxt(f_path, trks, delimiter=",")
                out_trk_all.update({fr_idx: trks})
            np.save(
                tnpy_all_path,
                out_trk_all,
            )
        else:
            print("{} tracking results already generated!!!".format(seq))
            print("Will not overwrite. If you want to overwrite, use flag --overwrite")
            continue

(1/6): 2373 frames
DETS SHAPE = (14, 7)
TRKS SHAPE = (14, 8)


/home/jupyter-dominik/AB3DMOT/AB3DMOT_libs/kitti_oxts.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function roty failed at nopython mode lowering due to: 
  @jit
/home/jupyter-dominik/.conda/envs/DAAV/lib/python3.8/site-packages/numba/core/utils.py:213: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-captured-errors
Exception origin:
  File "/home/jupyter-dominik/.conda/envs/DAAV/lib/python3.8/site-packages/numba/cpython/listobj.py", line 1131, in list_to_list
    assert fromty.dtype == toty.dtype

  warnings.warn(msg,
/home/jupyter-dominik/.conda/envs/DAAV/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "roty" was compiled in object mode without

DETS SHAPE = (14, 7)
TRKS SHAPE = (24, 8)
DETS SHAPE = (12, 7)
TRKS SHAPE = (20, 8)
DETS SHAPE = (10, 7)
TRKS SHAPE = (19, 8)
DETS SHAPE = (13, 7)
TRKS SHAPE = (18, 8)
DETS SHAPE = (11, 7)
TRKS SHAPE = (21, 8)
DETS SHAPE = (24, 7)
TRKS SHAPE = (26, 8)
DETS SHAPE = (16, 7)
TRKS SHAPE = (29, 8)
DETS SHAPE = (18, 7)
TRKS SHAPE = (16, 8)
DETS SHAPE = (19, 7)
TRKS SHAPE = (28, 8)
DETS SHAPE = (18, 7)
TRKS SHAPE = (30, 8)
DETS SHAPE = (20, 7)
TRKS SHAPE = (30, 8)
DETS SHAPE = (24, 7)
TRKS SHAPE = (35, 8)
DETS SHAPE = (17, 7)
TRKS SHAPE = (31, 8)
DETS SHAPE = (19, 7)
TRKS SHAPE = (33, 8)
DETS SHAPE = (28, 7)
TRKS SHAPE = (38, 8)
DETS SHAPE = (26, 7)
TRKS SHAPE = (39, 8)
DETS SHAPE = (23, 7)
TRKS SHAPE = (46, 8)
DETS SHAPE = (28, 7)
TRKS SHAPE = (36, 8)
DETS SHAPE = (25, 7)
TRKS SHAPE = (40, 8)
DETS SHAPE = (21, 7)
TRKS SHAPE = (39, 8)
DETS SHAPE = (22, 7)
TRKS SHAPE = (35, 8)
DETS SHAPE = (33, 7)
TRKS SHAPE = (34, 8)
DETS SHAPE = (30, 7)
TRKS SHAPE = (40, 8)
DETS SHAPE = (19, 7)
TRKS SHAPE = 

KeyboardInterrupt: 

In [ ]:
trks[0].shape